In [0]:
spark

Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql.functions import *
from configparser import ConfigParser

In [0]:
spark.conf.set("fs.azure.account.auth.type.raw97.dfs.core.windows.net","SharedKey")
spark.conf.set("fs.azure.account.key.raw97.dfs.core.windows.net","uVQzaztl+qd6C1/IiWwJF5y+6CiBtj5qbel4HCVPvVcuUBcvklhYW5pxMXprZOsT8+ndEfXSIYpA+AStBFiGFw==")

In [0]:
config = ConfigParser()

In [0]:
dbutils.fs.cp("abfss://silver@raw97.dfs.core.windows.net/configuration/configurations" , "file:///tmp/config.ini" )

Out[23]: True

In [0]:
config.read('/tmp/config.ini')

Out[24]: ['/tmp/config.ini']

In [0]:
inputpath = config['variables']['input']
outputpath = config['variables']['output']
jdbcurl=  config['variables']['jdbdurl']


In [0]:
#reading data from silver zone 
address =spark.read.parquet(inputpath +"SalesLT_Address.par")
customer=spark.read.parquet(inputpath +"SalesLT_Customer.par")
customerAddress=spark.read.parquet(inputpath +"SalesLT_CustomerAddress.par")
product=spark.read.parquet(inputpath +"SalesLT_Product.par")
productCategory=spark.read.parquet(inputpath +"SalesLT_ProductCategory.par")
productDescription=spark.read.parquet(inputpath +"SalesLT_ProductDescription.par")
productModel=spark.read.parquet(inputpath +"SalesLT_ProductModel.par")
pModelpDescription=spark.read.parquet(inputpath +"SalesLT_ProductModelProductDescription.par")
SalesOrderDetail=spark.read.parquet(inputpath +"SalesLT_SalesOrderDetail.par")
SalesOrderHeader=spark.read.parquet(inputpath +"SalesLT_SalesOrderHeader.par")

In [0]:
#reading empty tables from synapse 
SalesLTFact= spark.read.format("jdbc").option( "url",jdbcurl).option("dbtable","dbo.SalesLTFact").option("header",True).load()
AddressDim = spark.read.format("jdbc").option( "url",jdbcurl).option("dbtable","dbo.Addressdim").option("header",True).load()
Productdim =spark.read.format("jdbc").option( "url",jdbcurl).option("dbtable","dbo.Productdim").option("header",True).load()
Customerdim= spark.read.format("jdbc").option( "url",jdbcurl).option("dbtable","dbo.Customerdim").option("header",True).load()
SalesOrderdim=spark.read.format("jdbc").option("url",jdbcurl).option("dbtable","dbo.SalesOrderdim").option("header",True).load()

In [0]:
#registering synapse tables 
AddressDim.registerTempTable('Addressdim')
Productdim.registerTempTable('Productdim')
Customerdim.registerTempTable('Customerdim')
SalesOrderdim.registerTempTable('SalesOrderdim')

In [0]:
SalesLTFact.registerTempTable('SalesLTFact')
address.registerTempTable('address')
customerAddress.registerTempTable('customerAddress')
SalesOrderDetail.registerTempTable('SalesD')
customer.registerTempTable('customer')
product.registerTempTable('product')
SalesOrderHeader.registerTempTable('Sales')


In [0]:

spark.sql("insert into SalesLTFact(select  C.AddressID, C.CustomerID,P.ProductID,S.SalesOrderId,P.ProductModelID,P.StandardCost,P.ListPrice,P.Weight,d.OrderQty,d.UnitPrice,d.UnitPriceDiscount, d.LineTotal,S.SubTotal,S.TaxAmt,S.TotalDue from product as p inner join SalesD as d on d.ProductID = p.ProductID inner join Sales as S on S.SalesOrderId = d.SalesOrderId inner join customerAddress c on C.CustomerID = S.customerID)").show()

++
|
++
++

In [0]:
spark.sql("select * from SalesLTFact").show()

+---------+----------+---------+------------+--------------+------------+---------+--------+--------+---------+-----------------+-----------+-----------+---------+-----------+
AddressID|CustomerID|ProductID|SalesOrderID|ProductModelID|StandardCost|ListPrice| Weight|OrderQty|UnitPrice|UnitPriceDiscount| LineTotal| SubTotal| TaxAmt| TotalDue|
+---------+----------+---------+------------+--------------+------------+---------+--------+--------+---------+-----------------+-----------+-----------+---------+-----------+
 1086| 29485| 961| 71782| 36| 461.4448| 742.3500|13049.78| 3| 445.4100| 0.0000|1336.230000| 39785.3304|3182.8264| 43962.7901|
 653| 30113| 935| 71780| 62| 17.9776| 40.4900| 218.00| 2| 24.2940| 0.0000| 48.588000| 38418.6895|3073.4952| 42452.6519|
 653| 30113| 985| 71780| 23| 308.2179| 564.9900|12596.19| 1| 112.9980| 0.4000| 67.798800| 38418.6895|3073.4952| 42452.6519|
 1086| 29485| 955| 71782| 34| 1481.9379|2384.0700|11530.26| 1|1430.4420| 0.0000|1430.442000| 39785.3304|3182.8264| 43962.7901|
 1086| 29485| 966| 71782| 34| 1481.9379|2384.0700|11398.72| 2|1430.4420| 0.0000|2860.884000| 39785.3304|3182.8264| 43962.7901|
 653| 30113| 780| 71780| 20| 1265.6195|2319.9900|10781.83| 4|1391.9940| 0.0000|5567.976000| 38418.6895|3073.4952| 42452.6519|
 659| 29736| 979| 71784| 36| 461.4448| 742.3500|13213.08| 8| 445.4100| 0.0000|3563.280000|108561.8317|8684.9465|119960.8240|
 1086| 29485| 957| 71782| 34| 1481.9379|2384.0700|11747.98| 1|1430.4420| 0.0000|1430.442000| 39785.3304|3182.8264| 43962.7901|
 992| 29957| 875| 71783| 24| 3.3623| 8.9900| 0.00| 1| 5.3940| 0.0000| 5.394000| 83858.4261|6708.6741| 92663.5609|
 1019| 29653| 877| 71858| 119| 2.9733| 7.9500| 0.00| 1| 4.7700| 0.0000| 4.770000| 13823.7083|1105.8967| 15275.1977|
 653| 30113| 782| 71780| 20| 1251.9813|2294.9900|10591.33| 4|1376.9940| 0.0000|5507.976000| 38418.6895|3073.4952| 42452.6519|
 653| 30113| 867| 71780| 37| 26.1763| 69.9900| 0.00| 6| 41.9940| 0.0000| 251.964000| 38418.6895|3073.4952| 42452.6519|
 1086| 29485| 963| 71782| 36| 461.4448| 742.3500|13344.62| 1| 445.4100| 0.0000| 445.410000| 39785.3304|3182.8264| 43962.7901|
 1086| 29485| 967| 71782| 34| 1481.9379|2384.0700|11530.26| 2|1430.4420| 0.0000|2860.884000| 39785.3304|3182.8264| 43962.7901|
 992| 29957| 711| 71783| 33| 13.0863| 34.9900| 0.00| 15| 19.2445| 0.0500| 274.234125| 83858.4261|6708.6741| 92663.5609|
 1086| 29485| 954| 71782| 34| 1481.9379|2384.0700|11398.72| 1|1430.4420| 0.0000|1430.442000| 39785.3304|3182.8264| 43962.7901|
 992| 29957| 798| 71783| 29| 713.0798|1120.4900| 8060.29| 1| 672.2940| 0.0000| 672.294000| 83858.4261|6708.6741| 92663.5609|
 653| 30113| 991| 71780| 23| 294.5797| 539.9900|12759.49| 3| 323.9940| 0.0000| 971.982000| 38418.6895|3073.4952| 42452.6519|
 1086| 29485| 886| 71782| 10| 199.8519| 333.4200| 1451.49| 1| 200.0520| 0.0000| 200.052000| 39785.3304|3182.8264| 43962.7901|
 1086| 29485| 880| 71782| 107| 20.5663| 54.9900| 0.00| 9| 32.9940| 0.0000| 296.946000| 39785.3304|3182.8264| 43962.7901|
+---------+----------+---------+------------+--------------+------------+---------+--------+--------+---------+-----------------+-----------+-----------+---------+-----------+
only showing top 20 rows

In [0]:
spark.sql("insert into Addressdim(select AddressID,AddressLine1,AddressLine2 ,City,StateProvince,CountryRegion,PostalCode from address)").show()


++
|
++
++

In [0]:
spark.sql("insert into Productdim(select ProductID,ProductNumber,Size ,Weight,ProductCategoryID,ProductModelID from product)").show()


++
|
++
++

In [0]:
spark.sql("select * from Addressdim").show()

+---------+--------------------+------------+---------------+----------------+-------------+----------+
AddressID| AddressLine1|AddressLine2| City| StateProvince|CountryRegion|PostalCode|
+---------+--------------------+------------+---------------+----------------+-------------+----------+
 608| 25306 Harvey Rd.| NA|College Station| Texas|United States| 77840|
 510| 750 Philip Street| NA| Waterloo| Ontario| Canada| N2V|
 547|7000 Victoria Par...| NA| Toronto| Ontario| Canada| M4B 1V4|
 527| 252851 Rowan Place| NA| Richmond|British Columbia| Canada| V6B 3P7|
 496|Suite 500 995 W. ...| NA| Mississauga| Ontario| Canada| L5A 1H6|
 515| 9900 Ronson Drive| NA| Etobicoke| Ontario| Canada| M9W 3P3|
 457| 5250-505 Burning St| NA| Vancouver|British Columbia| Canada| V7L 4J4|
 461| 2560 Bay Street| NA| Toronto| Ontario| Canada| M4B 1V7|
 581| Po Box 8035996| NA| Dallas| Texas|United States| 75201|
 570| 52500 Liberty Way| NA| Fort Worth| Texas|United States| 76102|
 526|251420a 170th Street| NA| Edmonton| Alberta| Canada| T5|
 32| 26910 Indela Road| NA| Montreal| Quebec| Canada| H1Y 2H5|
 508|2900 - 25055 Dunn...| NA| Vancouver|British Columbia| Canada| V7L 4J4|
 462|630 University Av...| NA| Toronto| Ontario| Canada| M4B 1V7|
 503| P.O. Box 44000| NA| Winnipeg| Manitoba| Canada| R3|
 500| 5700 Explorer Drive| NA| Mississauga| Ontario| Canada| L4W 5J3|
 450| 575 Rue St Amable| NA| Quebec| Quebec| Canada| G1R|
 28| 9228 Via Del Sol| NA| Phoenix| Arizona|United States| 85004|
 524|9693 Louis H Lafo...| NA| Ville De'anjou| Quebec| Canada| J1G 2R3|
 606|99433 S. Greenbay...| NA| Racine| Wisconsin|United States| 53182|
+---------+--------------------+------------+---------------+----------------+-------------+----------+
only showing top 20 rows

In [0]:
spark.sql("insert into Customerdim(select CustomerID,Title,FirstName,MiddleName,LastName,Phone from customer)").show()


++
|
++
++

In [0]:
spark.sql("select * from Customerdim").show()

+----------+-----+---------+----------+--------+-------------------+
CustomerID|Title|FirstName|MiddleName|LastName| Phone|
+----------+-----+---------+----------+--------+-------------------+
 1| Mr.| Orlando| N.| Gee| 245-555-0173|
 91| Ms.| Joy| R.| Koski| 810-555-0198|
 198| Mr.| Payton| P.| Benson| 528-555-0183|
 18| Mr.| David| J.| Liu| 440-555-0132|
 23| Mr.| Kerim| NA| Hanif| 216-555-0122|
 209| Mr.| Robert| E.| Jones| 595-555-0131|
 48| Ms.| Anita| R.| Lucerne| 164-555-0118|
 3| Ms.| Donna| F.|Carreras| 279-555-0130|
 55| Mr.| Eric| NA| Lang| 932-555-0163|
 115| Mr.| George| Z.| Li| 699-555-0183|
 11| Ms.|Katherine| NA| Harding| 926-555-0159|
 129| Ms.| Connie| L.| Coffman| 426-555-0181|
 24| Mr.| Kevin| NA| Liu| 926-555-0164|
 7| Mr.| Dominic| P.| Gash| 192-555-0173|
 454| Ms.| Gail| NA|Erickson| 834-555-0132|
 97| Mr.| Eugene| NA| Kogan| 136-555-0134|
 43| Ms.| Elsa| NA| Leavitt| 482-555-0174|
 52| Ms.| Rebecca| NA| Laszlo|1 (11) 500 555-0155|
 76| Mr.| James| J.| Krow| 265-555-0190|
 110| Ms.| Kendra| N.|Thompson| 464-555-0188|
+----------+-----+---------+----------+--------+-------------------+
only showing top 20 rows

In [0]:
spark.sql("insert into SalesOrderdim(select SalesOrderID,SalesOrderDetailID,OrderQty from SalesD)").show()


++
|
++
++

In [0]:
spark.sql("select * from SalesOrderdim").show()

+------------+------------------+--------+
SalesOrderID|SalesOrderDetailID|OrderQty|
+------------+------------------+--------+
 71780| 110623| 4|
 71784| 110774| 8|
 71784| 110788| 7|
 71796| 111027| 7|
 71780| 110634| 3|
 71782| 110687| 3|
 71784| 110757| 8|
 71783| 110714| 9|
 71783| 110748| 11|
 71832| 111804| 1|
 71782| 110694| 1|
 71780| 110633| 2|
 71780| 110616| 4|
 71783| 110711| 6|
 71782| 110692| 2|
 71780| 110629| 2|
 71783| 110715| 7|
 71780| 110628| 1|
 71780| 110620| 1|
 71782| 110708| 6|
+------------+------------------+--------+
only showing top 20 rows

In [0]:
AddressdimG = sqlContext.table("Addressdim")
SalesLTFactG  = sqlContext.table("SalesLTFact")
ProductdimG = sqlContext.table("Productdim")
CustomerdimG = sqlContext.table("Customerdim")
SalesOrderdimG = sqlContext.table("SalesOrderdim")



In [0]:
#writin to Gold Zone 
SalesLTFactG.write.mode('overwrite').format('parquet').save(outputpath+'SalesLTFact_Table')
AddressdimG.write.mode('overwrite').format('parquet').save(outputpath+'AddressDimension_Table')
ProductdimG.write.mode('overwrite').format('parquet').save(outputpath+'ProductDimension_Table')
CustomerdimG.write.mode('overwrite').format('parquet').save(outputpath+'CustomerDimension_Table')
SalesOrderdimG.write.mode('overwrite').format('parquet').save(outputpath+'SalesOrderDimension_Table')

